In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.spatial.distance import cdist
from skimage import measure
from matplotlib import pyplot as plt
import os
import tifffile
from tqdm import tqdm
from typing import List, Tuple

In [2]:
selected_magnification = "20x"
selected_image_type = "raw"
max_dist = 9
aspect = np.array((1/3, 1/3))    # XY: 20x / aspect => 60x

### Read shift data

In [3]:
shifts = pd.read_csv(f"../5.0_comparing_magnifications/shift_correction/{selected_magnification}_{selected_image_type}.shifts.csv")
shifts.index = shifts["sid"].values
shifts.drop("sid", 1, inplace=True)

# Matching 20x_dw and reference dots

In [4]:
dots_data = pd.read_csv("../../data/magnifications_matching/20x_raw.field_thr.cleaned.all.tsv", sep="\t")

In [5]:
reference = pd.read_csv("../../data/60x_reference/ref__dw.field_thr.tsv", sep="\t")

In [6]:
def get_unassigned_count(pdist: np.ndarray) -> int:
    return np.sum(np.nansum(np.isnan(pdist), 1) != pdist.shape[1])

def match_dots(dotsA: np.ndarray, dotsB: np.ndarray) -> List[Tuple[int, int, float]]:
    pdist = cdist(dotsA, dotsB)
    
    matched: List[Tuple[int, int, float]] = []
    unassigned_rows = pdist.shape[0] + 1
    iteration_counter = 0
    print("Iteration #", end="")
    while unassigned_rows > get_unassigned_count(pdist) and 0 < get_unassigned_count(pdist):
        iteration_counter += 1
        print(f"...{iteration_counter}", end="")

        unassigned_rows = get_unassigned_count(pdist)
        A_id = 0
        while A_id < pdist.shape[0]:
            if pdist.shape[1] == np.sum(np.isnan(pdist[A_id, :])):
                A_id += 1
                continue
            B_id = np.nanargmin(pdist[A_id, :])
            if np.nanargmin(pdist[:, B_id]) == A_id:
                matched.append((A_id, B_id, pdist[A_id, B_id]))
                pdist[A_id, :] = np.nan
                pdist[:, B_id] = np.nan
            A_id += 1
    print("")
    return matched

In [7]:
match_output: List[pd.DataFrame] = []
match_output_full: List[pd.DataFrame] = []
match_counts: List[Tuple[int, int, int, int]] = []
for current_field_id in tqdm(np.unique(dots_data["sid"])):
    dots = dots_data.loc[current_field_id == dots_data["sid"], :].copy(
        ).sort_values("Value2", ascending=False).reset_index(drop=True)
    
    dot_coords = dots.loc[:, ("x", "y")].copy().reset_index(drop=True)
    dot_coords2 = dot_coords.copy() / aspect
    dot_coords2["x"] += (shifts.loc[current_field_id, "x"] * 9)
    dot_coords2["y"] += (shifts.loc[current_field_id, "y"] * 9)
    ref_coords = reference.loc[reference["sid"] == current_field_id, ("x", "y")].copy().reset_index(drop=True)
    
    matched: List[Tuple[int, int, float]] = match_dots(dot_coords2, ref_coords)
    
    matched_a = np.array(matched)
    matched_field = pd.DataFrame(dict(
        series=current_field_id,
        id_20x=matched_a[:, 0].astype("i"),
        id_60x=matched_a[:, 1].astype("i"),
        eudist=matched_a[:, 2]
    ))
    matched_field_within_dist = matched_field[matched_field["eudist"] <= max_dist]
    match_output.append(matched_field_within_dist)
    match_output_full.append(matched_field)
    match_counts.append((current_field_id,
                         matched_field_within_dist.shape[0],
                         matched_field.shape[0]-matched_field_within_dist.shape[0],
                         dot_coords2.shape[0],
                         ref_coords.shape[0]))

  0%|          | 0/7 [00:00<?, ?it/s]

Iteration #...1...2...3

 14%|█▍        | 1/7 [00:00<00:02,  2.28it/s]

...4...5...6...7
Iteration #...1...2...3...4...5...6...7...8...9...10...11...12

 29%|██▊       | 2/7 [00:01<00:03,  1.47it/s]

...


Iteration #...1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25

 86%|████████▌ | 6/7 [00:07<00:01,  1.35s/it]

...26...27...28...29...30...31...32...33...34...35...36...37
Iteration #...1...2...3...4...5...6...7

100%|██████████| 7/7 [00:07<00:00,  1.11s/it]

...8...9


In [8]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:, 1])
n_total_over_dist = np.sum(match_counts_a[:, 2])
n_total_20x = np.sum(match_counts_a[:, 3])
n_total_60x = np.sum(match_counts_a[:, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

6111/11190 (54.61%) 20x dots matched to 60x
6111/9944 (61.45%) 60x dots matched to 20x
3188 (28.49% 20x, 32.06% 60x) dots matched with distance > 9


In [9]:
for series_id in range(match_counts_a.shape[0]):
    n_matched = match_counts_a[series_id, 1]
    n_over_dist = match_counts_a[series_id, 2]
    n_20x = match_counts_a[series_id, 3]
    n_60x = match_counts_a[series_id, 4]
    print(f"""Field #{series_id+1}
    {n_matched}/{n_20x} ({n_matched/n_20x*100:.2f}%) 20x dots matched to 60x
    {n_matched}/{n_60x} ({n_matched/n_60x*100:.2f}%) 60x dots matched to 20x
    {n_over_dist} ({n_over_dist/n_20x*100:.2f}% 20x, {n_over_dist/n_60x*100:.2f}% 60x) dots matched with distance > {max_dist}
    """)

Field #1
    990/1554 (63.71%) 20x dots matched to 60x
    990/1272 (77.83%) 60x dots matched to 20x
    282 (18.15% 20x, 22.17% 60x) dots matched with distance > 9
    
Field #2
    1426/2249 (63.41%) 20x dots matched to 60x
    1426/1865 (76.46%) 60x dots matched to 20x
    439 (19.52% 20x, 23.54% 60x) dots matched with distance > 9
    
Field #3
    696/1291 (53.91%) 20x dots matched to 60x
    696/1032 (67.44%) 60x dots matched to 20x
    336 (26.03% 20x, 32.56% 60x) dots matched with distance > 9
    
Field #4
    1269/2201 (57.66%) 20x dots matched to 60x
    1269/1534 (82.72%) 60x dots matched to 20x
    265 (12.04% 20x, 17.28% 60x) dots matched with distance > 9
    
Field #5
    1142/2770 (41.23%) 20x dots matched to 60x
    1142/2471 (46.22%) 60x dots matched to 20x
    1329 (47.98% 20x, 53.78% 60x) dots matched with distance > 9
    
Field #6
    465/769 (60.47%) 20x dots matched to 60x
    465/869 (53.51%) 60x dots matched to 20x
    304 (39.53% 20x, 34.98% 60x) dots matche

#### General statistics, excluding field #7

In [10]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:6, 1])
n_total_over_dist = np.sum(match_counts_a[:6, 2])
n_total_20x = np.sum(match_counts_a[:6, 3])
n_total_60x = np.sum(match_counts_a[:6, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

5988/10834 (55.27%) 20x dots matched to 60x
5988/9043 (66.22%) 60x dots matched to 20x
2955 (27.28% 20x, 32.68% 60x) dots matched with distance > 9


#### General statistics, excluding fields #6-7

In [11]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:5, 1])
n_total_over_dist = np.sum(match_counts_a[:5, 2])
n_total_20x = np.sum(match_counts_a[:5, 3])
n_total_60x = np.sum(match_counts_a[:5, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

5523/10065 (54.87%) 20x dots matched to 60x
5523/8174 (67.57%) 60x dots matched to 20x
2651 (26.34% 20x, 32.43% 60x) dots matched with distance > 9


### Export

In [12]:
matched_dots = pd.concat(match_output)

In [13]:
matched_dots.to_csv(os.path.join("../../data/magnifications_matching",
                                 f"{selected_magnification}_{selected_image_type}.matched_dots.field_thr.clean.tsv"
                                ), sep="\t", index=False)

In [14]:
pd.DataFrame(match_counts_a, columns=["series_id", "matched", "matched_over_dist", "raw_total", "dw_total"]
            ).to_csv(os.path.join("../../data/magnifications_matching",
                                  f"{selected_magnification}_{selected_image_type}.matched_dots.field_thr.counts.clean.tsv"
                                 ), sep="\t", index=False)